<a href="https://colab.research.google.com/github/venkat2ram/ML_From_Scratch/blob/master/NeuralNet_From_Scratch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


In [0]:
def init_params(layer_dims):
  np.random.seed(3)
  L=len(layer_dims)
  params={}
  for l in range(1,L):
    params['w'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])/0.01
    params['b'+str(l)]=np.zeros((layer_dim[l],1))
  return params


In [0]:
def sigmoid(Z):
  A=1/(1+np.exp(np.dot(-1,Z)))
  cache=(Z)
  return A,cache

In [0]:
def forward_prop(X,params):
  A=X
  caches=[]
  L=len(params)//2
  for l in range(1,L+1):
    A_prev=A
    Z=np.dot(params['w'+str(l)],A_prev)+params['b'+str(l)]
    linear_cache=(A_prev,params['w'+str(l)],params['b'+str(l)])
    A,activation_cache=sigmoid(Z)
    cache=(linear_cache,activation_cache)
    caches.append(cache)
  return A,caches   


In [0]:
def cost_function(A, Y):
  m=Y.shape[1]
  cost=(-1/m)*np.sum(np.dot(np.log(A),Y.T)+np.dot(np.log(1-A),1-Y.T))
  return cost

**Rough**

In [73]:
a=np.array([[0.2,0.2,0.3],[0.3,0.4,0.5],[0.4,0.3,0.5]])
b=np.array([[0.3,0.4,0.5],[0.6,0.7,0.8])
cost_function(a,b)


4.742705654642286

In [88]:
np.divide(b,y_hat)+np.divide(1-b,1-y_hat)

array([[7.36647463e+32, 9.82196618e+32, 1.22774577e+33],
       [6.45269158e+53, 7.52814018e+53, 8.60358877e+53]])

In [55]:
a.shape[1]

3

In [26]:
 np.random.seed(3)
np.random.randn(2,2)

array([[ 1.78862847,  0.43650985],
       [ 0.09649747, -1.8634927 ]])

In [31]:
a=[1,2]
b=[[3,4],[5,6]]
c=[[3],[5]]
ca=(a,b)
ca=(ca,c)
ca

(([1, 2], [[3, 4], [5, 6]]), [[3], [5]])

In [57]:
for i in range(1,6):
  print(i)

1
2
3
4
5


In [0]:
layer_dim=[3,4,4,2]

In [0]:
params=init_params(layer_dim)

In [0]:
y_hat,cache=forward_prop(a,params)

In [86]:
y_hat

array([[4.07250435e-34, 4.07250435e-34, 4.07250435e-34],
       [9.29844535e-55, 9.29844535e-55, 9.29844535e-55]])

In [87]:
cache

[((array([[0.2, 0.2, 0.3],
          [0.3, 0.4, 0.5],
          [0.4, 0.3, 0.5]]),
   array([[ 178.86284734,   43.65098505,    9.64974681],
          [-186.34927034,  -27.73882025,  -35.47589793],
          [  -8.27414815,  -62.70006768,   -4.3818169 ],
          [ -47.72180304, -131.38647534,   88.46223805]]),
   array([[0.],
          [0.],
          [0.],
          [0.]])),
  array([[ 52.72776371,  56.12788753,  80.30922013],
         [-59.78185931, -59.00815155, -87.51214019],
         [-22.21757669, -28.04940177, -36.02318673],
         [-13.57540799, -35.56027933, -35.77865955]])),
 ((array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
          [1.08910157e-26, 2.36094242e-26, 9.86189595e-39],
          [2.24403197e-10, 6.58111667e-13, 2.26635940e-16],
          [1.27137793e-06, 3.60053398e-16, 2.89418032e-16]]),
   array([[  88.13180422,  170.95730637,    5.00336422,  -40.46774146],
          [ -54.53599476, -154.64773156,   98.23674343, -110.10676301],
          [-118.504

In [72]:
params['w1']

array([[ 178.86284734,   43.65098505,    9.64974681],
       [-186.34927034,  -27.73882025,  -35.47589793],
       [  -8.27414815,  -62.70006768,   -4.3818169 ],
       [ -47.72180304, -131.38647534,   88.46223805]])

In [74]:
a

array([[0.2, 0.2, 0.3],
       [0.3, 0.4, 0.5],
       [0.4, 0.3, 0.5]])

In [75]:
np.dot(params['w1'],a)

array([[ 52.72776371,  56.12788753,  80.30922013],
       [-59.78185931, -59.00815155, -87.51214019],
       [-22.21757669, -28.04940177, -36.02318673],
       [-13.57540799, -35.56027933, -35.77865955]])

In [82]:
sigmoid(2)

(0.8807970779778823, 2)